# APMTH 207: Advanced Scientific Computing: 
## Stochastic Methods for Data Analysis, Inference and Optimization
## Group Project
**Harvard University**<br>
**Spring 2018**<br>
**Instructors: Rahul Dave**<br>
**Assigned Teaching Staff: Patrick 'Likestogiveas' Ohiomoba**<br>
**Due Date: ** Friday, April 27th, 2018 at 11:59pm<br><br>
**Assigned Paper:** Bayesian Incremental Learning for Deep Neural Networks (https://arxiv.org/abs/1802.07329)<br>
**Group Members:** Alex Truong, David Loving, Jonathan Lee


## Background
Background information relevant to the paper


In many practical applications of machine learning, data are not fully collected prior to model training but arrive in chunks, with accrual possibly over a long time period. For instance, a machine learning model for medical diagnoses may accomodate an initial set of patient data, but the model parameters may need to be further tuned as yet more clinical data are generated. Thus, a method to tweak predictive models using new data would be broadly applicable for a number of applications of machine learning.

Deep learning, or deep neural networks, represents a family of nonparametric machine learning models which have been gaining widespread use due to their flexibility and predictive capabilities in data analysis and artificial intelligence. However, deep learning models are marked by complex parameterization, making it challenging to tune these models. Furthermore, training of these models is prone to reach a suboptimal solution, especially .

Kochurov et al. describe a method to update pre-existing deep learning models with new data using stochastic variational inference. Here we summarize the relevant theory underlying this method, provide a python implementation of this method, and benchmark our implementation against other AM207 models with the MNIST dataset.

something like this?

## Overview of methods
A detailed summary of the relevant methods, math and procedure

The paper is pretty short and centers around a single algorithm, explained in detail on page 2 and summarized on page 5. The algorithm depends on performing approximate variational inference, for which the authors discuss a number of techniques. We will focus primarily on using one of those (fully factorized gaussian approximation) and address others if time permits. The key demonstration is to take a pre-trained conventional DNN and update it with new data. The authors have done this on MNIST and CIFAR-10, and we plan to show how to replicate their results on MNIST, as well as attempting a non-image task if time permits.

Need to add the math stuff...

The main algorithm is as follows, as shown in Appendix A of Kochurov et al.:
<img src="algorithm.png">

Formally (adapted from various parts of Kochurov et al.), we begin with a dataset we define to be $\mathcal D$ that is broken up into $T$ parts, $\mathcal D = \{\mathcal D_1,...\mathcal D_T\}$. Each of these parts arrives sequentially during training. We implement a model that will train on the first $t-1$ parts ($\mathcal D_1,...\mathcal D_{t-1}$, then retrain it on the $\mathcal D_t$-th part without have access to the first $t-1$ parts and taking into account dependencies.

We utilize a pre-trained conventional DNN and create a Gaussian prior to initialize the Bayesian network, weights centered around the pre-trained means, with standard deviations fixed.

For each chunk of data that arrives, we sample from it and update our parameters (e.g. SGD or Adam - Kochurov et al. uses Adam with default settings).


Kochurov et al. also make several approximations which help to increase their model performance in the paper - laplace approximation and stochastic variation inference. We briefly introduce both of these topics below, as well as a bit of the underlying mathematics that is being done.

### Tangent 1 - Laplace Approximation

To determine the parameters for initializing the algorithm (creation of the bayesian model from standard DNN), We already know to use the trained weights as the initial means, but we also need to find the starting variance as well. Kochurov et al. propose either grid search or Laplace approximation, and ultimately implement Laplace approximation in their experiments.

Formally presented, Laplace's method is used to approximate integrals of a particular form

$$\int_a^be^{Mf(x)}dx$$

where $f(x)$ is a twice-differentiable function, $M$ is some large number, and $a$ and $b$ are potentially infinite.
The reference that Kochurov et al. uses (Azevedo-filho, 1994) presents the integral formulation in the form

$$\int_a^b\! h(x) e^{M g(x)}\, dx$$

which Wikipedia helpfully expands:

$$\int_a^b\! h(x) e^{M g(x)}\, dx\ \approx\ \sqrt{\frac{2\pi}{M|g''(x_0)|}} h(x_0) e^{M g(x_0)} \ \text { as } M\to\infty \,$$

(These are functionally identical - $g(x)$ must be twice-differentiable, and it is clearer why this is the case in the above expansion.) In the application outlined in Kochurov et al., Laplace's method is used to approximate parameters of posterior distributions - specifically, variance. The actual - and more helpful - approximation that is taking place in Kochurov et al. uses equation (14) from Azevedo-filho, 1994:

$$\mathrm{Var}(g|X) \approx \hat{\mathrm{Var}}_n(g|X)(1+O(n^{-2})$$

where $n$ is simply the n-th weight that we are estimating, and $O$ represents the order of the error terms. Effectively, we just compute the variance of the sampling of each batch of $D$, $n$ and assume that to represent the variance of the $n$-th slice.

### Tangent 2 - Stochastic Variational Inference

In many cases the posterior distribution is intractable, so approximation must be done in more creative ways. The authors decide upon stochastic variational inference to approximate each weight. Stochastic variational inference is the application of stochastic optimization to variational inference (Hoffman et al., 2013).

#### Subtangent 1 - Variational Inference

In variational inference, the posterior distribution $P(\textbf Z|\textbf X)$ of some set of unknown variables $\textbf Z = \{Z_1,...Z_n\}$ with some observed data $\textbf X$ is approximated by a variational distribution, $Q(\textbf Z)$. $Q(\textbf Z)$ is restricted to the family of distributions that are simpler than $P(\textbf Z|\textbf X)$. The (lack of) similarity between Q and P is determined with a dissimiliarity function, $d(Q;P)$, and variational inference serves to minimize this function. A typical dissimiliarity function is Kullback–Leibler divergence (KL-divergence). It is defined as

$$D_{\mathrm{KL}}(Q || P) = \sum_\mathbf{Z}  Q(\mathbf{Z}) \log \frac{Q(\mathbf{Z})}{P(\mathbf{Z}\mid \mathbf{X})}$$

Minimizing a function like this "turns the inference problem into an optimization problem." (Blei et al., 2018)

#### Subtangent 2 - Stochastic Optimization

Instead of operating on the entire slice of data we are provided, we randomly sample from it (minibatches), and take statistics from those subsets of the slice, and repeat.

### Tangent 2, continued - Stochastic Variational Inference

Stochastic variational inference applies random sampling of $\mathbf{X}$, and updating modeling parameters accordingly. Once the minimization of the dissimilarity function is stabilized (converged), we arrive at our approximation of the posterior distribution.

All put together, our primary aim is to create a "worst-case scenario" version of the model, that implements the simplest facets of the multiple options provided in the paper (e.g. basic pre-train network, straightforward Bayesian network, with fully factorized gaussian approximation), and go on to explore alternative network structures and compare test accuracy. Our decisions are illustrated in the Implementation section below. Additionally, we completely ignore the posterior approximation, and set starting standard deviations to 1 for added simplicity.

## Implementation
A python implementation of selected important methods from the paper (It is perfectly fine to find methods and code elsewhere on the web, as long as you internalize, understand, implement, and explain what is going on)

Might make sense to try to get our hands on the code from these guys...

Sent an email to Dmitry Vetrov...let's see if he responds...

## Method Application
A demonstration of the results of those methods on
1. at-least one dataset (from the paper if available)
2. if the paper does not provide a dataset, use a simulated dataset chosen by your group
3. you may choose to use do both of the above anyways; simulated datasets are the best way to understand the main thrust as well as the edge cases of an algorithm.

MNIST

## Method Comparison
An analysis contrasting your results reimplemented from the paper with a standard method from AM207 or your area of expertise (e.g. mcmc vs special snowflake mcmc)

Use some algorithm from class? (probably the MLP stuff)

## Bibliography

To be added here...

Adriano Azevedo-filho. Laplace’s method approximations for probabilistic inference in belief networks
with continuous variables. In In de Mantaras, pp. 28–36. Morgan Kaufmann, 1994.

Matt Hoffman, David M. Blei, Chong Wang, and John Paisley. Stochastic variational inference,
2012.